In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from utils.helpers import md

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")
pipe(messages)

Device set to use mps:0


RuntimeError: MPS backend out of memory (MPS allocated: 6.76 GB, other allocations: 628.00 KB, max allowed: 6.77 GB). Tried to allocate 64.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

### Local Run Llama 3.2 1B Using 🤗

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically maps layers to CPU or GPU
    offload_folder="./offload"  # Folder for offloaded layers
).to(device)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/jack.yangjy/.pyenv/versions/3.10.11/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jack.yangjy/.pyenv/versions/3.10.11/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/jack.yangjy/projects/youtube_summarizer/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jack.yangjy/projects/youtube_summarizer/venv/lib/python3.10/site-

My MacBook can only handle max 1B parameter model and prompt size ~20 words

In [50]:
prompt = "What is the capital of France? Answer in one word"
inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to(device)

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=20, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What is the capital of France? Answer in one word: Paris.

(Note: I'm assuming you want the capital of France, not the capital of the


### Llama Using Hosted API Replicate

In [76]:
def chunk_transcript(cleaned_transcript: list[str], chunk_size=50_000) -> list[str]:
    size = 0
    chunk = []
    lst_chunk = []
        
    for i in range(len(cleaned_transcript)):
        segment = cleaned_transcript[i]
        if size + len(segment) > chunk_size:
            lst_chunk.append(' '.join(chunk))
            chunk = []
            size = 0
        chunk.append(segment)
        size += len(segment)
    if not lst_chunk:
        return ' '.join(chunk)
    return lst_chunk

In [ ]:
# link = 'https://www.youtube.com/watch?v=139UPjoq7Kw&t=25s&ab_channel=JaneStreet'
link = 'https://www.youtube.com/watch?v=ZvSNcnG2eqY&t=82s&ab_channel=2and20'
from data.get_youtube_data_OLD import fetch_transcript

output = fetch_transcript(link)
transcript = output['transcript']

from data.preprocess import clean_transcript, chunk_transcript

cleaned = clean_transcript(transcript)
chunks = chunk_transcript(cleaned)
chunks

Language: English
Language: en
Generated automatically: False
Translation available: True
-----


"0s - October 19th, 2015 election night in Canada, 5s - dissatisfied with the direction of the country. 7s - Record numbers of Canadians came out to vote. 10s - The result. 11s - Justin Trudeau is elected Prime Minister 14s - of Canada, with a large crowd cheering his name. 18s - Trudeau had one simple message. 20s - It's time for a change in this country, my friends. 24s - A real change. 28s - Canadians across the country were excited and energized. 31s - They believed in a bright future for Canada. 34s - The government would build more homes, make life 37s - more affordable and usher Canada into a golden age. 41s - The Canadian dream. 43s - But fast forward to today, and that dream has faded. 46s - Housing prices have increased at record rates. 50s - Food banks are turning away hungry Canadians, 53s - and immigrants are leaving in record numbers. 56s - The Canadian dream has become a nightmare. 59s - This is the dark side of Canada. 62s - If you talk to real Canadians, more often tha

In [31]:
transcript[15:]

[{'text': 'Housing prices have increased\nat record rates.',
  'start': 46.254,
  'duration': 3.003},
 {'text': 'Food banks are turning away\nhungry Canadians,',
  'start': 49.758,
  'duration': 2.877},
 {'text': 'and immigrants\nare leaving in record numbers.',
  'start': 52.635,
  'duration': 2.878},
 {'text': 'The Canadian dream\nhas become a nightmare.',
  'start': 55.513,
  'duration': 3.003},
 {'text': 'This is the dark side\nof Canada.',
  'start': 58.641,
  'duration': 3.003},
 {'text': 'If you talk to real Canadians,\nmore often than not,',
  'start': 61.686,
  'duration': 2.586},
 {'text': 'they will tell you\nhow hopeless',
  'start': 64.272,
  'duration': 1.502},
 {'text': 'they feel about their future.', 'start': 65.774, 'duration': 1.584},
 {'text': "They'll share how,\nwhen growing up, they dreamed",
  'start': 67.358,
  'duration': 1.836},
 {'text': 'of a life raising a family\nin a comfortable home',
  'start': 69.194,
  'duration': 2.669},
 {'text': 'and in a safe nei

In [3]:
import os
from dotenv import load_dotenv
import replicate

load_dotenv(override=True)
api_token = os.getenv("REPLICATE_API_TOKEN")

def llama3_8b(prompt):
    replicate_client = replicate.Client(api_token=api_token)
    output = replicate_client.run(
      "meta/meta-llama-3-8b-instruct",
      input={"prompt": prompt}
    )
    return ''.join(output)

In [32]:
prompt = f'help me summarize this youtube video transcript. Transcript has the format "timestamp s - text". Show the timestamp for important topics: {chunks}'
output = llama3_8b(prompt)
md(output)



Here is a summarized version of the YouTube video transcript, highlighting important topics with timestamps:

**Election Night 2015**
(0s) October 19th, 2015 election night in Canada, where Justin Trudeau is elected Prime Minister.

**Housing Affordability**
(10s) The result: record numbers of Canadians came out to vote, electing Justin Trudeau as Prime Minister. (14s) Trudeau's message: "It's time for a change in this country, my friends. A real change." (18s) But, fast-forward to today, and the Canadian dream has faded.

**The Dark Side of Canada**
(46s) Housing prices have increased at record rates, food banks are turning away hungry Canadians, and immigrants are leaving in record numbers. (56s) The Canadian dream has become a nightmare.

**Housing Market**
(74s) The average home price in Canada is out of reach for most people, with a family needing to earn $170,000 per year to buy a home. (84s) In Vancouver, the number jumps to $360,000 per year.

**Government Inaction**
(230s) The government has not taken sufficient action to address the housing crisis, with many arguing that they are making life harder for Canadians.

**Immigration and Housing**
(250s) The government's immigration policy has contributed to the housing crisis, with nearly 3 million immigrants admitted into the country since 2016, leading to increased demand for housing.

**Underinvestment in Housing**
(328s) Canada needs 5 million more homes by 2030 to achieve affordability targets, but developers agree that this is impossible due to a lack of investment.

**Money Laundering and Housing**
(420s) Canada is an attractive place for money laundering, with a report concluding that significant amounts of black market money is buying and selling properties in Canada.

**Food Prices and Inflation**
(451s) Food prices have risen by over 10% since 2022, leading to a record high of nearly 2 million Canadians visiting food banks in March 2023.

**Carbon Tax and Inflation**
(522s) The carbon tax has made the cost of gas and heating more expensive, with Canadians spending more on gas than any other G7 country.

**Telecommunications and Banking**
(584s) The Canadian wireless market is the most expensive in the world, and banking fees are high due to a lack of competition.

**Economic Decline**


In [22]:
md('<br>'.join(cleaned))

0s - October 19th, 2015 election night in Canada,<br>5s - dissatisfied with the direction of the country.<br>7s - Record numbers of Canadians came out to vote.<br>10s - The result.<br>11s - Justin Trudeau is elected Prime Minister<br>14s - of Canada, with a large crowd cheering his name.<br>18s - Trudeau had one simple message.<br>20s - It's time for a change in this country, my friends.<br>24s - A real change.<br>28s - Canadians across the country were excited and energized.<br>31s - They believed in a bright future for Canada.<br>34s - The government would build more homes, make life<br>37s - more affordable and usher Canada into a golden age.<br>41s - The Canadian dream.<br>43s - But fast forward to today, and that dream has faded.<br>46s - Housing prices have increased at record rates.<br>50s - Food banks are turning away hungry Canadians,<br>53s - and immigrants are leaving in record numbers.<br>56s - The Canadian dream has become a nightmare.<br>59s - This is the dark side of Canada.<br>62s - If you talk to real Canadians, more often than not,<br>64s - they will tell you how hopeless<br>66s - they feel about their future.<br>67s - They'll share how, when growing up, they dreamed<br>69s - of a life raising a family in a comfortable home<br>72s - and in a safe neighborhood.<br>74s - Or with home prices soaring.<br>75s - This dream has become a fantasy<br>78s - to buy the average home in Canada.<br>79s - A family needs to earn $170,000 per year.<br>84s - And in Vancouver that number jumps dramatically<br>86s - to $360,000 per year.<br>89s - Compare this to the median family income of $70,000.<br>93s - And it becomes clear<br>95s - that it is nearly impossible to buy a home in Canada.<br>98s - Most Canadians who don't own a home have given up and don't<br>101s - see it as remotely possible in their lifetime.<br>104s - The sad part about this<br>106s - Canada's housing wasn't always this expensive.<br>109s - In fact, homeownership was possible for almost everyone.<br>112s - But in the last few decades,<br>114s - housing prices have increased so much faster than wages<br>117s - that it has forced citizens<br>119s - to abandon the hope of homeownership.<br>121s - Even if you can afford a home,<br>123s - you aren't getting very much.<br>125s - Take Vancouver, for example.<br>127s - The average home<br>127s - here is nearly $2 million, and it looks like this.<br>132s - Who wants to spend<br>133s - nearly $2 million on a small home like this?<br>136s - When you can get a mansion in places like Austin<br>139s - for the same amount?<br>140s - Now, not everyone wants to buy a home, and many<br>143s - are perfectly fine renting.<br>145s - But renting is just as hard as buying a home due to years<br>149s - of under building NIMBYism, population<br>152s - growth and government red tape.<br>154s - The supply of rental units is far outweighed by demand<br>158s - Across Canada,<br>159s - rental vacancy sits at an all time low of 1.5%.<br>164s - This means people are fighting to find a place to live.<br>167s - Each application for an apartment is met<br>169s - with hundreds of competing applications,<br>171s - and renters are left feeling hopeless.<br>174s - It's getting to the point<br>174s - where you can choose to keep a roof over your head,<br>177s - or you can choose to keep the lights on and eat.<br>178s - But you can't do both.<br>179s - The low levels of vacancy<br>181s - are leading to skyrocketing prices.<br>184s - Landlords realize that renters have few options,<br>187s - and therefore they continue to raise rents<br>189s - to extract maximum value.<br>191s - Our landlord increased our rent by 7000 per month.<br>195s - The outcome?<br>197s - Rents continue to rise uncontrollably<br>200s - and will do so for years to come.<br>202s - For example, the average one bedroom apartment in Toronto<br>205s - and Vancouver is nearly $3,000.<br>208s - After taxes and rent, Canadians barely<br>210s - have enough money to make ends meet.<br>212s - People are skipping meals, taking up multiple jobs,<br>215s - living in their cars, and more.<br>217s - Debt has become an absolute necessity to make ends meet.<br>220s - And Canada now has the third<br>222s - highest household debt to GDP ratio on Earth.<br>226s - All of this is compounded<br>227s - by the inability of the government to help.<br>230s - In fact,<br>231s - many would argue that the government<br>233s - is making life much harder for Canadians.<br>236s - The problem with housing affordability is a combination<br>239s - of federal and local government policies<br>241s - and inaction on a federal level.<br>244s - Prime Minister Justin Trudeau has overseen<br>246s - the largest increase in Canadian real estate prices<br>249s - in history.<br>250s - Much of this is due to his immigration policy.<br>253s - Since 2016, the government has admitted<br>255s - nearly 3 million immigrants into the country.<br>258s - That is a record and 40% higher per year<br>262s - in the last few decades.<br>263s - To make matters worse, there are over 1 million<br>266s - foreign students in the country.<br>268s - By increasing the population so significantly<br>271s - in such a short period of time.<br>273s - The government has squeezed<br>274s - an already under built housing market.<br>277s - It is simple economics.<br>278s - Increasing demand without increasing supply<br>281s - causes prices to go up.<br>283s - Furthermore, most immigrants are looking<br>285s - for entry level homes and apartments,<br>288s - placing a disproportionate amount of pressure<br>290s - on average, and young Canadians<br>292s - who are looking for a starter property.<br>294s - For example, many students at UBC<br>296s - can no longer find housing<br>298s - and now live in vans and parking lots to survive.<br>301s - I have to wait,<br>302s - I guess like more than a year to get housing<br>305s - on campus in UBC.<br>306s - One student even flies from a different city<br>309s - because it's cheaper than rent.<br>311s - Rooming houses are also commonplace<br>314s - across the country,<br>315s - where landlords squeeze<br>316s - five students into one bedroom to maximize profit.<br>319s - Immigration isn't the only reason<br>321s - for rising prices, though,<br>323s - and it's easy for folks to point the finger.<br>325s - But Canada has underinvested in housing for decades.<br>328s - In fact, CIBC estimates that Canada needs 5 million<br>332s - more homes by 2030 to achieve affordability targets.<br>337s - Developers agree this is impossible.<br>340s - Of course,<br>340s - you can't flip the switch overnight to build homes.<br>343s - And in Canada, local governments<br>345s - have shown an unwillingness to invest in housing.<br>348s - For one, citizens who own homes are often<br>351s - against development<br>352s - as it would lower the value of their properties.<br>354s - Permit approvals are hard to get,<br>356s - and NIMBYism challenges most large developments.<br>359s - Elected officials know this<br>361s - and therefore don't push hard for housing to avoid losing<br>363s - an election.<br>364s - Furthermore, each city and province levies<br>367s - a multitude of taxes<br>368s - that drive the cost and timeline for development.<br>371s - In some cities,<br>372s - this can add over 20% to the cost of a home,<br>375s - and this cost is directly<br>377s - passed on to consumers by developers.<br>379s - A part of the reason taxes and red<br>381s - tape continue to increase<br>383s - is the government is growing faster than the population.<br>386s - From 2020 to 2023.<br>388s - The B.C. government grew by 23%.<br>391s - Compare this to the private sector, which grew by only 1%.<br>395s - This explosive job growth requires more taxes.<br>399s - A similar pattern can be seen in the federal government,<br>401s - which has added nearly $20 billion a year in costs<br>404s - since 2015, and the final nail<br>407s - in the coffin for housing affordability.<br>409s - Foreign Investment<br>410s - Canada is an extremely attractive<br>412s - and safe place to invest your money,<br>414s - but it also happens to be one of the easiest places<br>417s - in the OECD to launder money.<br>420s - Another bombshell in the ongoing money<br>422s - laundering scandal in B.C.,<br>423s - a report commissioned by the government<br>425s - agreed and concluded<br>426s - that significant amounts of black market money<br>429s - was buying<br>430s - and selling properties in Canada,<br>431s - especially Vancouver and Toronto.<br>434s - But what has the government done to stop this?<br>436s - Nothing.<br>437s - Money laundering is just as easy as ever.<br>440s - Now housing is just one component of this nightmare.<br>443s - What about everyday costs Canadians face?<br>446s - Canada saw food prices rise by over 10% since 2022,<br>451s - leading nearly<br>451s - 2 million Canadians to visit food banks in March of 2023,<br>455s - a record high and double the 2019 levels.<br>459s - One of the reasons Canadian<br>460s - food prices are so high is the lack of competition.<br>463s - But grocery chain profits are not the reason<br>467s - for food inflation.<br>468s - A handful of companies own nearly 80% market share.<br>473s - The government has historically done<br>474s - nothing to prevent food industry consolidation,<br>477s - which has led to a highly concentrated market<br>479s - that is able to extract maximum profits.<br>482s - The worst<br>482s - example of this is bread,<br>484s - which is dominated by only two players.<br>486s - Not only this,<br>487s - but the government has assisted<br>489s - in increasing food prices in Canada<br>491s - by protecting the poultry, egg and dairy industries.<br>494s - This alone cost the average Canadian family<br>497s - nearly $450 per year.<br>500s - That could be the difference between making rent or not.<br>503s - The federal plan<br>504s - puts a price on pollution that brings down emissions,<br>507s - and it spurs innovation on the kind of investments<br>511s - in emissions reduction that we need.<br>512s - While at the same time putting more money<br>516s - in the pockets of Canadians, this story of government<br>518s - policies leading to higher prices is nothing new.<br>521s - For example,<br>522s - the government<br>522s - introduced a carbon tax which makes the cost of gas<br>525s - and heating more expensive than ever.<br>527s - Canadians spend more on gas than any other G7 country<br>531s - by a long shot.<br>532s - The carbon tax has been met<br>533s - with significant pushback from citizens and politicians.<br>537s - In fact,<br>537s - Liberal MPs have spoken out against the carbon tax<br>540s - as the number of complaints<br>542s - from constituents has risen dramatically.<br>544s - This came to a head in the Atlantic provinces<br>546s - when citizens could no longer bear<br>548s - the cost of the carbon tax.<br>550s - Given the use of expensive and very dirty fuel oil,<br>553s - the Liberal government conceded and dropped the tax<br>556s - in an effort to not lose support.<br>558s - Recently, the government has pushed the<br>560s - narrative that most citizens are better off<br>563s - in spite of carbon taxes due to rebates.<br>566s - However,<br>567s - the parliamentary budget officer<br>568s - clarified and said that citizens are worse off<br>571s - if you include the economic implications.<br>573s - But this isn't just limited to carbon taxes.<br>576s - Canadians experience higher prices across<br>579s - multiple industries because of the government.<br>581s - The most glaring example is telecommunications.<br>584s - Here's a quick question.<br>585s - Are we paying too much for cell phone<br>587s - and internet bills here in Canada?<br>588s - Absolutely.<br>589s - The Canadian wireless market is the most expensive<br>592s - in the world,<br>593s - seven times more expensive in Australia, and 1000 times<br>596s - more expensive than Finland.<br>598s - The craziest part is that the fourth largest<br>600s - player, Shaw, was recently acquired<br>602s - by the largest player in the country.<br>604s - Rogers and the government did nothing to stop it.<br>607s - In most countries,<br>608s - such a deal would most likely be stopped or delayed.<br>611s - But in Canada,<br>612s - major corporations<br>613s - have extreme influence and often<br>615s - get away with these anti-competitive transactions.<br>618s - Banking is similar.<br>619s - There is no competition nor options for consumers<br>622s - to start up.<br>623s - A bank is nearly impossible in Canada due to tight<br>626s - regulatory controls that favor large entities.<br>630s - This has led to Canadians paying some of the highest<br>632s - banking fees in the developed world.<br>634s - It is estimated that Canadians pay $250 more per year<br>638s - on banking fees than peers in the UK and Australia.<br>641s - Why? No competition.<br>644s - Now I could list a million more examples,<br>647s - but the conclusion is the same.<br>649s - Canada is extremely expensive.<br>651s - One way to counter this is to grow<br>653s - the economy and introduce high paying jobs.<br>656s - However, Canada has struggled to do this.<br>658s - Since 2019, Canada has seen a decline<br>661s - in economic productivity.<br>663s - While the US has seen a 6% increase.<br>666s - This decline<br>667s - is partly responsible for stagnant wages in Canada<br>670s - and reduction in the quality of life.<br>672s - One of the drivers for this decline is lack of private<br>675s - sector investment.<br>676s - Canadian businesses are investing<br>678s - outside of Canada more than ever before.<br>680s - But why aren't companies investing in Canada?<br>683s - Simply put,<br>684s - they believe they can generate greater returns elsewhere,<br>686s - especially as they lose faith<br>688s - in the current Canadian regime,<br>690s - which is not seen as business friendly.<br>692s - Another reason is lack of competition.<br>695s - Most major Canadian industries<br>697s - are controlled by a few key players.<br>699s - These players have no real incentive to invest,<br>702s - leading to reductions in productivity.<br>704s - The US has low levels of market<br>706s - concentration across industries,<br>708s - leading to constant reinvestment and innovation.<br>710s - As companies continue to seek a competitive edge,<br>713s - another reason is government intervention.<br>716s - For example, Canada has seen a major decline<br>718s - in investment in the energy sector<br>720s - due to federal prices that penalize the industry.<br>723s - Since 2014, energy investment has declined<br>726s - by nearly $60 billion per year.<br>729s - But one could argue that oil and gas is the past,<br>732s - and Canada should focus on future proofing its economy.<br>735s - Which is absolutely fair.<br>736s - However, since 2016, GDP derived from technology<br>740s - companies has only grown with inflation,<br>743s - meaning it hasn't really grown at all.<br>745s - All efforts by the government to incentivize<br>747s - industrial investment have failed.<br>749s - But this is not simply limited<br>751s - to large businesses and money outflow.<br>753s - Canada is seeing an exodus of startups and top talent<br>757s - like never before.<br>758s - Each year, 7% of Canadians go south of the border<br>762s - searching for opportunity.<br>763s - And why wouldn't they?<br>764s - Jobs pay more.<br>766s - Cost of living is lower and the US is much more dynamic.<br>769s - The loser in this scenario is not the government,<br>772s - but hard working Canadians whose lives continue to become<br>775s - more difficult as productivity continues to decline.<br>779s - The central bank will have less leeway<br>780s - to maintain lower interest rates,<br>782s - further challenging cost of living.<br>784s - Overall, the outlook does not look great.<br>788s - Let's hope that something changes soon.<br>790s - Subscribe to 2&20 for more business and economics insight.

## Full

In [37]:
import os
from dotenv import load_dotenv
import replicate

load_dotenv(override=True)
api_token = os.getenv("REPLICATE_API_TOKEN")

def llama3_8b(prompt):
    replicate_client = replicate.Client(api_token=api_token)
    output = replicate_client.run(
      "meta/meta-llama-3-8b-instruct",
      input={
          "max_tokens": 2000,
          "prompt": prompt
          }
    )
    return ''.join(output)

def llama3_70b(prompt):
    replicate_client = replicate.Client(api_token=api_token)
    output = replicate_client.run(
      "meta/meta-llama-3-70b-instruct",
      input={
          "max_tokens": 2000,
          "prompt": prompt
          }
    )
    return ''.join(output)

True

In [51]:
from data.get_youtube_data import YouTubeDataFetcher
from data.preprocess import get_prompt

video_url = 'https://www.youtube.com/watch?v=QlUcyaXiXPE&ab_channel=MorningBrewDaily'
# video_url = 'https://www.youtube.com/watch?v=9U48NlbOzCU&ab_channel=MLEpath'

youtube = YouTubeDataFetcher()
meta_data = youtube.get_meta_data(video_url)

prompt = get_prompt(meta_data=meta_data)
print(prompt)

2024-12-22 16:18:42,003 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-12-22 16:18:42,941 - INFO - Language: English
2024-12-22 16:18:42,942 - INFO - Language: en
2024-12-22 16:18:42,944 - INFO - Generated automatically: False
2024-12-22 16:18:42,945 - INFO - Translation available: True
2024-12-22 16:18:42,945 - INFO - -----
2024-12-22 16:18:42,946 - INFO - Found manual English transcript



Instructions:
* First provide video title, jump straight into it no need leading message
* Provide high level overall summary.
* For each chapter create an informative and concise summary in **bullet-point format**, provide additional context where appropriate.
* Include the chapter's timestamp, infer start and end duration.

Can refer to this example structure:
```
Title:
    High level summary
Chapter 1: Example Title (00:00 - 02:00)
    - Summary

Chapter 2: Example Title (02:00 - 04:00)
    - Summary
```

Inputs:
Title: Tech CEOs Charm Trump With Big Bucks
Channel Name: Morning Brew Daily
Transcript is auto-generated: False

(Timestamp: 00:00) Patterson bookstore bonus extravaganza
Transcript:
Good Morning Brew Daily Show. I'm Neal Freyman. And I'm Toby Howell. today. It's not Barbie. It's not Lego. The best selling toy in the world is. Yeah. You're going to have to wait for that then. It's a bird. It's a plane. No, it's a drone hovering over new Jersey. But in reality, it's proba

In [46]:
output = llama3_8b(prompt)
md(output)

2024-12-22 14:20:35,777 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"




Here is the summary of the podcast episode:

**Title:** Tech CEOs Charm Trump With Big Bucks

**High-level summary:** The podcast discusses various news stories, including James Patterson's bonus to independent bookstores, tech CEOs donating to Trump's inauguration fund, Broadcom's stock price surge, and the mysterious drone sightings in New Jersey.

**Chapter 1: Patterson bookstore bonus extravaganza (00:00 - 02:15)**

* James Patterson, author of popular books, gave $500 holiday bonuses to 600 employees at independent bookstores across the US.
* The bonus is a tradition started by Patterson in 2015, and it's a heartwarming story of generosity and giving back.

**Chapter 2: Tech CEOs donate to Trump (02:15 - 6:40)**

* Tech CEOs, including Sundar Pichai, Tim Cook, Mark Zuckerberg, and Jeff Bezos, are donating to Trump's inauguration fund.
* The donations are seen as a way to curry favor with the new administration and ensure their interests are heard.
* The podcast discusses the potential impact of these donations on the tech industry.

**Chapter 3: Broadcom quietly cashin' in (6:40 - 10:45)**

* Broadcom's stock price has surged, making it a $1 trillion company.
* The company's success is attributed to its AI hardware and software business.
* The podcast discusses the company's CEO, Hock Tan, and his acquisition strategy.

**Chapter 4: What is up with drones?? (10:45 - 15:10)**

* There have been reports of drones flying over New Jersey, causing concern and confusion.
* The podcast discusses the potential causes of the sightings, including hobbyist drones and the fact that it's legal to fly drones at night.
* The FBI has stated that most of the reported sightings are actually planes, not drones.

**Chapter 5: Hot Wheels lead the way (15:10 - 18:30)**

* Hot Wheels is the best-selling toy in the world, with fans of all ages.
* The brand has been successful due to its affordability, nostalgia, and collector's market.
* The podcast discusses the company's licensing strategy and its connection to Barbie.

**Chapter 6: Week Ahead (18:30 - 21:30)**

* The Federal Reserve is holding its final meeting of the year, and it's expected to cut interest rates.
* The podcast discusses the potential impact of the rate cut on the economy and the stock market.
* There are also several new movie releases and TV shows to look out for this week.

I hope this summary helps! Let me know if you have any further questions.

In [43]:
output = llama3_70b(prompt)
md(output)

2024-12-22 14:12:54,631 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-70b-instruct/predictions "HTTP/1.1 201 Created"




**Tech CEOs Charm Trump With Big Bucks**

High-level summary: The video discusses how top tech CEOs are trying to build relationships with President-elect Trump, donating to his inaugural fund and meeting with him at Mar-a-Lago. The CEOs are trying to ensure they are on good terms with the incoming administration, which will have a significant impact on their businesses.

**Chapter 1: James Patterson's Generosity (00:00 - 02:00)**
- James Patterson handed out $500 holiday bonuses to 600 employees at independent bookstores across the country.
- He has sold over 425 million copies of his books and has a total income of $700 million over the last decade.

**Chapter 2: Tech CEOs Donate to Trump (02:15 - 06:40)**
- Tech CEOs like Sundar Pichai, Tim Cook, Mark Zuckerberg, and Jeff Bezos are meeting with Trump at Mar-a-Lago and donating $1 million to his inaugural fund.
- This is a common practice for CEOs to build relationships with the incoming administration, which sets policies that affect their businesses.
- Elon Musk's close relationship with Trump has raised concerns among other CEOs, who fear he may use his influence to harm their businesses.

**Chapter 3: Broadcom's Success (06:40 - 10:45)**
- Chipmaker Broadcom has reached a $1 trillion market cap, driven by its AI hardware and software business.
- The company supplies advanced AI hardware to big tech companies like Google and Meta, and its AI revenue is up 220% this year.
- Broadcom's CEO, Hock Tan, expects the company's AI opportunity to reach $90 billion by 2027.

**Chapter 4: The Drone Mystery (10:45 - 15:10)**
- Mysterious drone sightings have been reported in New Jersey, with 11 confirmed instances of drones illegally entering a U.S. Army research facility's airspace.
- Lawmakers are calling for more transparency from federal officials, who say the drones don't appear to be linked to foreign governments.
- Some experts believe the sightings may be due to hobbyist drones or misidentification of planes.

**Chapter 5: Trump Wants to End Daylight Saving Time (15:10 - 18:30)**
- President-elect Trump wants to end Daylight Saving Time, calling it "inconvenient and very costly to our nation."
- The debate around Daylight Saving Time has been ongoing, with some arguing it saves energy and boosts consumer spending, while others say it disrupts sleep patterns and health.

**Chapter 6: Hot Wheels Lead the Way (18:30 - 21:30)**
- Hot Wheels is the best-selling toy in the world, with sales growing 14% to reach nearly $1.5 billion last year.
- The brand's success is attributed to its affordability, nostalgia, and a major collector scene among adults.
- Hot Wheels is also licensing its IP, which is expected to maximize revenue.

**Chapter 7: Week Ahead (21:30 - End)**
- The Fed is expected to cut rates at its final meeting of the year on Wednesday.
- The week ahead also includes the release of Mr. Beast's Squid Game-inspired competition show on Amazon Prime Video and the premiere of Mufasa, the prequel to The Lion King, in theaters.